<a href="https://colab.research.google.com/github/marwankashef/NLP/blob/master/Scraping_Tweets_from_Twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy
import pandas as pd
import nltk
import spacy
import tweepy
import json
import math
import time

#nltk.download('vader_lexicon')
#!python -m spacy download en_core_web_lg
#!python -m spacy download en_core_web_md

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from scipy import spatial
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn import metrics

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [ ]:
def login():
    # TODO: Insert your keys and tokens here
    consumer_key  = 'gCsCBJVh15Z7blGcLGbWl3zCb'
    consumer_secret  = 'cBOMt32b9bbhKcItvd67TPlTKhzjQaDYWXHkw4TpiLY0daJKx1'
    access_token  = '1261725119912738819-Y3RnF4DSWrYknIsvcgbhHP6KOGrfKX'
    access_token_secret  = 'NQkoP9OzAGl9PTHSieslZgcaHrzcxPZroIRiLjlD822WR'
    
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth, wait_on_rate_limit=True)
    
    return api

def twitter_search(api, text_query, count=100, **kw):
  tweets = api.search(q=text_query, count=count, **kw)
        
  tweets_list = []
  for tweet in tweets:
      tweets_list.append((tweet.id, tweet.created_at,tweet.user.screen_name,tweet.text))
        
  df = pd.DataFrame(list(tweets_list), columns=['ID', 'Time', "Screen Name", "Text"])
  return df

In [ ]:
api = login()

In [ ]:
# Put whatever you want as text query
tweets_df = twitter_search(api, text_query="@Tesla")
tweets_df.head()

,ID,Time,Screen Name,Text
0,1270773340685434882,2020-06-10 17:42:28,Mujahid1_,@Chrom_Jockstrap @ihavehugepp @juancarIosbg @L...
1,1270773337606758400,2020-06-10 17:42:28,hayatsmma,RT @ElonsWorld: Tesla is now officially the mo...
2,1270773309895032833,2020-06-10 17:42:21,Daleb678gmailc1,RT @GerberKawasaki: Dear @elonmusk and @Tesla ...
3,1270773261564104708,2020-06-10 17:42:10,decosta_tucker,RT @BoredEscapeGoat: @PPathole @elonmusk @Tesl...
4,1270773199912001536,2020-06-10 17:41:55,dissolvedB123,RT @elonmusk: @GerberKawasaki @Tesla (Formerly...


In [ ]:
all_df = pd.DataFrame(columns=['ID', 'Time', "Screen Name", "Text"])

# Put whatever you want here
text_query = '@Tesla'


for _ in range(1): # How many times do you want to do this?
    new_tweets = twitter_search(api, text_query, since_id=all_df.ID.max())
        
    print(len(new_tweets))
    if len(new_tweets) > 0:
        all_df = all_df.append(new_tweets, ignore_index=True)

    time.sleep(0) #Sleep for 15 minutes

100


In [ ]:
all_df.info()
all_df.head()
all_df.ID.nunique()
all_df.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   ID           100 non-null    object        
 1   Time         100 non-null    datetime64[ns]
 2   Screen Name  100 non-null    object        
 3   Text         100 non-null    object        
dtypes: datetime64[ns](1), object(3)
memory usage: 3.2+ KB


,ID,Time,Screen Name,Text
0,1270773868643393536,2020-06-10 17:44:34,valleysue,@donnyosmond @Tesla Ouch! Glad you`re ok. Kudo...
1,1270773841766182914,2020-06-10 17:44:28,2TermPresDT,RT @donnyosmond: My @Tesla had a messy encount...
2,1270773793234079747,2020-06-10 17:44:16,sturgilln,RT @GerberKawasaki: Dear @elonmusk and @Tesla ...
3,1270773734027276289,2020-06-10 17:44:02,geoffcarmichael,@donnyosmond @Tesla It’s handled the semitruck...
4,1270773732429135876,2020-06-10 17:44:02,LibrarianPanduh,RT @JenniferJoyOK: @gtbynum @Tesla @elonmusk M...


100

,ID,Time,Screen Name,Text
95,1270771780647600129,2020-06-10 17:36:16,Der_Engineer,@elonmusk @elonmusk @Tesla congrats to be the ...
96,1270771745813860353,2020-06-10 17:36:08,jluedke67,@donnyosmond @Tesla Glad everyone is ok!
97,1270771720312496134,2020-06-10 17:36:02,mazzahamish,@donnyosmond @Tesla 😱😱😱
98,1270771705884139525,2020-06-10 17:35:59,shubham_arora_0,RT @elonmusk: @GerberKawasaki @Tesla (Formerly...
99,1270771703174574081,2020-06-10 17:35:58,reallytalljock1,RT @elonmusk: @GerberKawasaki @Tesla (Formerly...


In [ ]:
all_df.to_csv('tesla.csv')

In [ ]:
from google.colab import files

In [ ]:
files.download('tesla.csv')